In [1]:

#Para visualizacion del susuario
def inttopeso(a):
    a=str(a)
    a=a[:-6]+'.'+a[-6:-3]+'.'+a[-3:]
    a='$'+a[1:] if len(a)<9 else '$'+a
    return a
def facturapdf(cliente,Factura,remision,cotizacion):
    with open('info_compra.csv', "r") as csvfile:
        data = list(csv.reader(csvfile))

    elements = []
    
    styles = getSampleStyleSheet()
    styleNormal = styles['Normal']

    if not remision:
        line1 = 'MONICA ROLDAN ORTEGA'
        line2 = 'NIT: 59.680.231-6'
    else:
        line1=''
        line2=''
    line3 = 'Fecha: {}'.format(datetime.datetime.now().strftime("%d-%m-%y"))
    line4 = 'Cliente: {}'.format(cliente)
    if remision:
        outfilename = 'RemisionN°'+str(Factura)+'_'+datetime.datetime.now().strftime("%d-%m-%y")+'_'+cliente.replace(' ','_')+ '.pdf'
        outfiledir = '\\Remisiones\\'
    if not remision and not cotizacion:
        outfilename = 'FacturaN°'+str(Factura)+'_'+datetime.datetime.now().strftime("%d-%m-%y")+'_'+cliente.replace(' ','_')+ '.pdf'
        outfiledir = '\\Facturas\\'
    if cotizacion:
        line5 = '<font color="red" size=20>COTIZACIÓN</font>'
        logo="Cabecera_cotizacion.png"
        lineF='Esta oferta es válida por 10 días, los precios están sujetos a cambios sin previo aviso. Los equipos cuentan con garantía de 12 meses. Elementos eléctricos no tienen garantía. Las garantías no cubren: aparatos quemados por descargas eléctricas y/o manipulados por terceros y/o con golpes por caídas.'
        im = Image(logo, 8.33 * inch, 3.33 * inch,hAlign="CENTER")
        outfilename = 'CotizacionN°'+str(Factura)+'_'+datetime.datetime.now().strftime("%d-%m-%y")+'_'+cliente.replace(' ','_')+ '.pdf'
        outfiledir = '\\Cotizaciones\\'
    else:
        lineF='Los equipos cuentan con garantía de 12 meses en marcas Hilook, Hikvision, Dahua. Cámaras genéricas 3 meses. Elementos eléctricos no tienen garantía. Las garantías no cubren: aparatos quemados por descargas eléctricas y/o manipulados por terceros y/o con golpes por caídas.'
        line5= '<font color="red">Factura N°: {}</font>'.format(Factura)
        logo="Cabecera_cotizacion.png"
        im = Image(logo, 8.33 * inch, 3.33 * inch,hAlign="CENTER")
    if remision:
        line5= '<font color="red">Remision N°: {}</font>'.format(Factura)
    elements.append(im)
    elements.append(Paragraph(line1, styleNormal))
    elements.append(Paragraph(line2, styleNormal))
    elements.append(Spacer(inch, .25 * inch))
    elements.append(Paragraph(line3, styleNormal))

    elements.append(Paragraph(line4, styleNormal))
    style_right = ParagraphStyle(name='right', parent=styles['Normal'], alignment=TA_RIGHT)
    elements.append(Paragraph(line5,style_right))
    elements.append(Spacer(inch, .25 * inch))

    all_cells = [(0, 0), (-1, -1)]
    header = [(0, 0), (-1, 0)]
    column0 = [(0, 0), (0, -1)]
    column1 = [(1, 0), (1, -1)]
    column2 = [(2, 0), (2, -1)]
    column3 = [(3, 0), (3, -1)]
    column4 = [(4, 0), (4, -1)]
    column5 = [(5, 0), (5, -1)]
    column6 = [(6, 0), (6, -1)]
    table_style = TableStyle([
        ('VALIGN', all_cells[0], all_cells[1], 'TOP'),
        ('LINEBELOW', header[0], header[1], 1, colors.black),
        ('ALIGN', column0[0], column0[1], 'LEFT'),
        ('ALIGN', column1[0], column1[1], 'LEFT'),
        ('ALIGN', column2[0], column2[1], 'LEFT'),
        ('ALIGN', column3[0], column3[1], 'RIGHT'),
        ('ALIGN', column4[0], column4[1], 'RIGHT'),
        ('ALIGN', column5[0], column5[1], 'LEFT'),
        ('ALIGN', column6[0], column6[1], 'RIGHT'),
    ])

    
    colWidths = [
        0.7 * cm,  # Column 0
        2.9 * cm,  # Column 1
        3.2 * cm,  # Column 2
        0.7 * cm,  # Column 3
        2.5 * cm,  # Column 4
        7.2 * cm,  # Column 5
        2.1 * cm,  # Column 6
    ]


    for index, row in enumerate(data):
        for col, val in enumerate(row):
            if col != 5 or index == 0:
                data[index][col] = val.strip("'[]()")
            else:
                data[index][col] = Paragraph(val, styles['Normal'])
            if col==1:
                data[index][col] = Paragraph(val, styles['Normal'])


    t = Table(data, colWidths=colWidths)
    t.setStyle(table_style)
    elements.append(t)
    
    elements.append(Spacer(inch, 1.25 * inch))
    elements.append(Paragraph(lineF, styleNormal))
    outfilepath = os.path.dirname(os.path.abspath('Facturas/'))+outfiledir+outfilename


    archivo_pdf = SimpleDocTemplate(
        outfilepath,
        pagesize=letter,
        rightMargin=40,
        leftMargin=40,
        topMargin=40,
        bottomMargin=28)
    archivo_pdf.build(elements)

In [2]:
def csvcreador(cantidad,descuento,codes,instalador,FPrecio,IVA,Celda0=[]):
    products=[]
    N=len(cantidad)
    for _ in codes:
        Row=np.array(Arch.loc[Arch['ID']==_])
        Row=Row.reshape([8])
        if Row[3]=='AGOTADA':
            N+=-1
        else:
            products+=[Row]
    write=[]
    Total=0
    ST=0
    VTWD=0
    for i in range(0,N):
        IDs=codes[i]
        ref=products[i][1]
        Vlru=products[i][3]
        
        if cantidad[i]==4 and products[i][5]!='SOLO DETAL':
            Vlru=products[i][5]
            
        if cantidad[i]==5 and products[i][6]!='SOLO DETAL':
            Vlru=products[i][6]
        
        if cantidad[i]>=6 and products[i][7]!='SOLO DETAL':
            Vlru=products[i][7]
        
        if cantidad[i]<=3 and instalador:
            Vlru=products[i][4]
        
        if cantidad[i]<=3 and not instalador:
            Vlru=products[i][3]
        
        if IDs>=438 and cantidad[i]>=50 and products[i][5]!='SOLO DETAL':
            Vlru=products[i][5]
        
        if IDs>=438 and cantidad[i]>=100 and products[i][6]!='SOLO DETAL':
            Vlru=products[i][6]
        if IDs>=438 and cantidad[i]<50 and products[i][6]!='SOLO DETAL' and instalador:
            Vlru=products[i][4]
        
        if IDs>=438 and cantidad[i]<50 and products[i][6]!='SOLO DETAL' and not instalador:
            Vlru=products[i][3]
        
        if FPrecio:
            Vlru=products[i][3]
        
        if FPrecio and instalador:
            Vlru=products[i][4]
            
        if IVA:
            Vlru=products[i][4]
    
        C=cantidad[i]
        D=descuento[i]*100
        Des=products[i][2]
        ST+=int(Vlru*C)
        VTWD+=int(Vlru*descuento[i]*C)
        VT=int(Vlru*C*(1-descuento[i]))
        Total+=VT
        VT=str(VT)
        VT=VT[:-6]+'.'+VT[-6:-3]+'.'+VT[-3:]
        VT='$'+VT[1:] if len(VT)<9 else '$'+VT
        line=np.array([IDs,ref,inttopeso(int(Vlru)),C,str(int(D))+'%',Des,VT])
        write.append(line)
    
    if len(Celda0)!=0:
        altotal=0
        for _ in Celda0:
            altotal+=int(int(_[1])*int(_[2])*(1-int(_[3])*1e-2))
            VTWD+=int(int(_[1])*int(_[2])*(int(_[3])*1e-2))
            ST+=int(int(_[1])*int(_[2]))
    else:
        altotal=0
    
    with open('info_compra.csv', mode='w') as employee_file:
        employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        employee_writer.writerow(['ID','REFERENCIA','VLR\nUNITARIO','CANT','DESCUENTO','DESCRIPCION','VALOR TOTAL \nCON DESCUENTO'])
        for _ in write:
            employee_writer.writerow(_)
        if Celda0!=0:
            for j,_ in enumerate(Celda0):
                Final=str(int(int(_[1])*int(_[2])*(1-int(_[3])*1e-2)))
                Final=Final[:-6]+'.'+Final[-6:-3]+'.'+Final[-3:]
                Final='$'+Final[1:] if len(Final)<9 else '$'+Final
                employee_writer.writerow([j,'-',inttopeso(str(int(_[1]))),str(int(_[2])),str(int(_[3]))+'%',_[0],Final])
        if IVA:
            PIVA=str(int((Total+altotal)*0.19))
            PIVA=PIVA[:-6]+'.'+PIVA[-6:-3]+'.'+PIVA[-3:]
            PIVA='$'+PIVA[1:] if len(PIVA)<9 else '$'+PIVA
            SubT=str(Total+altotal)
            SubT=SubT[:-6]+'.'+SubT[-6:-3]+'.'+SubT[-3:]
            SubT='$'+SubT[1:] if len(SubT)<9 else '$'+SubT
            Total=str(int(Total*1.19)+altotal)
            Total=Total[:-6]+'.'+Total[-6:-3]+'.'+Total[-3:]
            Total='$'+Total[1:] if len(Total)<9 else '$'+Total
            employee_writer.writerow(['','','','','','','DESCUENTOS: \t {} \n SUBTOTAL:\t {} \n IVA:\t {} \n TOTAL:\t {} '.format(inttopeso(VTWD),SubT,PIVA,Total)])
        else:
            Total=str(int(Total)+altotal)
            Total=Total[:-6]+'.'+Total[-6:-3]+'.'+Total[-3:]
            Total='$'+Total[1:] if len(Total)<9 else '$'+Total
            employee_writer.writerow(['','','','','','','SUBTOTAL: {} \n DESCUENTOS: {} \n TOTAL: {}'.format(inttopeso(ST),inttopeso(VTWD),Total)])

In [ ]:
from tkinter import *
import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np
import os
import csv
import datetime
from reportlab.lib.units import cm, inch
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import *
from reportlab.lib.enums import TA_JUSTIFY, TA_LEFT, TA_CENTER, TA_RIGHT
from reportlab.lib.styles import getSampleStyleSheet,ParagraphStyle
Arch=pd.read_excel('Info.xlsx')
Arch["DESCRIPCION"]=Arch["DESCRIPCION"].str.replace("\n", " ")
Arch["DESCRIPCION"]=Arch["DESCRIPCION"].str.replace("/", " ")

root = Tk()
root.title('Digital Shop')
#bg=PhotoImage(file='bg.png',master=root)
#Fondo=Label(root,image=bg)
#Fondo.place(x=0,y=0,relwidth=1,relheigh=1)
photo = PhotoImage(file='logo-2.png',master=root)
image_label = ttk.Label(
    root,
    image=photo,
    padding=5,
    compound='center'
)
image_label.grid(column=1,row=0)

from idlelib.tooltip import OnHoverTooltipBase

class ComboboxTip(OnHoverTooltipBase):
    def __init__(self, combobox_widget, hover_delay=1000):
        super(ComboboxTip, self).__init__(combobox_widget, hover_delay=hover_delay)
        self.tips = {}
        self._current_item = 0

        combobox_widget.tk.eval("""
proc callback {y} {
     event generate %(cb)s <<OnMotion>> -y $y
}

set popdown [ttk::combobox::PopdownWindow %(cb)s]
bind $popdown.f.l <Motion> {callback %%y}
""" % ({"cb": combobox_widget}))

        self._id4 = combobox_widget.bind("<<OnMotion>>", self._on_motion)

    def _on_motion(self, event):
        current_item = int(self.anchor_widget.tk.eval("$popdown.f.l nearest %i" % event.y))
        if current_item != self._current_item:
            self._current_item = current_item
            self.hidetip()
            if current_item in self.tips:
                self.schedule()
            else:
                self.unschedule()

    def __del__(self):
        try:
            self.anchor_widget.unbind("<<OnMotion>>", self._id4)
        except TclError:
            pass
        super(ComboboxTip, self).__del__()

    def add_tooltip(self, index, text):
        self.tips[index] = text

    def get_position(self):
        """choose a screen position for the tooltip"""
        try:
            h = self.anchor_widget.winfo_height()
            bbox = self.anchor_widget._getints(self.anchor_widget.tk.eval("$popdown.f.l bbox %i" % self._current_item))
            #print(bbox[2])
            return bbox[0] + 325, bbox[1] + bbox[-1] + h
        except Exception:
            return 20, self.anchor_widget.winfo_height() + 1

    def showcontents(self):
        label = Label(self.tipwindow, text=self.tips[self._current_item], justify=tk.LEFT,
                         background="#ffffe0", relief=tk.SOLID, borderwidth=1,wraplength = 300)
        
        label.grid()



def search():
    value_to_search = var.get()
    
    if value_to_search == "" or value_to_search == " ":
        dim_combo['values'] = element_names
    
    else:
        value_to_siplay = []
        
        for value in element_names:
            
            if value_to_search in value:
                value_to_siplay.append(value)
        dim_combo['values'] = value_to_siplay
        for j,_ in enumerate(value_to_siplay):
            t.add_tooltip(j,_)

def select(event):
    myLabel=Label(root,text=dim_combo.get()[:45]+'...').pack()
code=[]
cantidad=[]
descuento=[]
info_nm=[]
info_0=[]
instalador=BooleanVar()
def agregar():
    code.append(int(np.array(Arch.loc[Arch['DESCRIPCION']==dim_combo.get()])[0][0]))
    cantidad.append(int(can.get()))
    descuento.append(int(Des.get())*10**-2)
    info_nm.append(dim_combo.get()[:45]+'...'+'\t'+can.get()+'\t'+Des.get())
Celda0=[]
def agregarcelda0():
    Celda0.append([Celda01.get(),Celda02.get(),Celda03.get(),Celda04.get()])
    info_0.append(Celda01.get()[:45]+'...'+'\t'+Celda03.get()+'\t'+Celda04.get()+'\t'+Celda02.get())
                
def restart_program():
    """Restarts the current program.
    Note: this function does not return. Any cleanup action (like
    saving data) must be done before calling this function."""
    python = sys.executable
    os.execl(python, python, * sys.argv)
    
def factura():
    csvcreador(cantidad,descuento,code,instalador.get(),FPrecio.get(),IVA.get(),Celda0=Celda0)
    cotizacion=Cotizacion.get()
    remision=Remision.get()
    
    with open('enumerador.csv',mode='r') as f:
        h=list(csv.reader(f))
        fa=int(h[0][0])
        Re=int(h[0][1])
        Co=int(h[0][2])
    f.close()
    
    if remision:
        Factura=Re
        
        with open('enumerador.csv',mode='w') as f:
            f.write(f'{fa},{Re+1},{Co}')
        f.close()
    
    if cotizacion:
        Factura=Co
        
        with open('enumerador.csv',mode='w') as f:
            f.write(f'{fa},{Re},{Co+1}')
        f.close()
    
    if not remision and not cotizacion:
        Factura=fa
        
        with open('enumerador.csv',mode='w') as f:
            f.write(f'{fa+1},{Re},{Co}')
        f.close()
    
    facturapdf(Client.get(),Factura,remision,cotizacion)

def ver():
    top= Toplevel(root)
    top.geometry("750x250")
    top.title("Productos")
    from tkinter import Frame
    main_frame=Frame(top)
    main_frame.pack(side=LEFT, fill=BOTH, expand=1)

    my_canvas=Canvas(main_frame)
    my_canvas.pack(side=LEFT, fill=BOTH, expand=1)

    my_scrollbar=ttk.Scrollbar(top, orient=VERTICAL, command=my_canvas.yview)
    my_scrollbar.pack(side=RIGHT, fill=Y)
    def on_mousewheel(event):
        scroll = -1 if event.delta > 0 else 1
        my_canvas.yview_scroll(scroll, "units")
    my_canvas.configure(yscrollcommand=my_scrollbar.set)
    my_canvas.bind('<Configure>', lambda e: my_canvas.configure(scrollregion=my_canvas.bbox('all')))
    my_canvas.bind("<MouseWheel>", on_mousewheel)
    second_frame=Frame(my_canvas)
    def deletenm(m):
        del code[m]
        del cantidad[m]
        del descuento[m]
        del info_nm[m]
        top.destroy()
        ver()
    def delete0(m):
        del Celda0[m]
        del info_0[m]
        top.destroy()
        ver()
    my_canvas.create_window((0,0),window=second_frame, anchor='nw')

    for i,_ in enumerate(info_nm):
        Label(second_frame,text=_).grid(column=0,row=i)
        Button(second_frame,text='Borrar', command= lambda m=i :deletenm(m)).grid(column=1,row=i)
    Label(second_frame,text='----------------Agregados----------------').grid(column=0,columnspan=2)
    if len(info_nm)==0:
        i=0
    for j,_ in enumerate(info_0):
        Label(second_frame,text=_).grid(column=0,row=j+2+i)
        Button(second_frame,text='Borrar', command= lambda m=j :delete0(m)).grid(column=2,row=j+2+i)
    

element_names = list(np.array(Arch["DESCRIPCION"]))
#Val=[]
#for _ in element_names:    
#    Val.append(_[:45])

dim_combo = ttk.Combobox(root,width=50,values=element_names)
#dim_combo['values'] = element_names
#dim_combo.bind("<<ComboboxSelected>>", select)
dim_combo.grid(row=1,columnspan=3)
t = ComboboxTip(dim_combo)

for j,_ in enumerate(element_names):
    t.add_tooltip(j,_)

var=StringVar()
entry = Entry(root, textvariable=var,width=25)
entry.grid(column=0,row=2,columnspan=2)

search_button = Button(root, text="BUSCAR", command=search)
search_button.grid(column=1,row=2)

can=StringVar()
can.set(1)
CanLabel=Label(root,text='CANTIDAD:')
can_s=Entry(root, textvariable=can)
CanLabel.grid(column=0,row=3)
can_s.grid(column=1,row=3)



Des=StringVar()
Des.set('0')
DesLabel=Label(root,text='DESCUENTO:')
Des_s=Entry(root, textvariable=Des)
DesLabel.grid(column=0,row=4)
Des_s.grid(column=1,row=4)



Enter_button=Button(root, text="AGREGAR", command=agregar)
Enter_button.grid(column=1,row=5)

Client=StringVar()
ClienLabel=Label(root,text='NOMBRE CLIENTE:')
Client_s=Entry(root, textvariable=Client)
ClienLabel.grid(column=0,row=6)
Client_s.grid(column=1,row=6)

C1 = Checkbutton(root, text = "INSTALADOR", variable=instalador, \
                 onvalue = True, offvalue = False, height=0, \
                 width = 20)
C1.grid(column=0,row=7)

Remision=BooleanVar()

C2 = Checkbutton(root, text = "REMISION", variable=Remision, \
                 onvalue = True, offvalue = False, height=0, \
                 width = 20)
C2.grid(column=1,row=7)

Cotizacion=BooleanVar()
C3 = Checkbutton(root, text = "COTIZACION", variable=Cotizacion, \
                 onvalue = True, offvalue = False, height=0, \
                 width = 20)
C3.grid(column=2,row=7)

FPrecio=BooleanVar()
C4=Checkbutton(root,text='FORZAR PRECIO', variable=FPrecio, \
              onvalue=True, offvalue=False, height=0, \
              width=20)
C4.grid(column=0,row=8)

IVA=BooleanVar()
C5=Checkbutton(root,text='INCLUIR IVA', variable=IVA, \
              onvalue=True, offvalue=False, height=0,\
              width=20)
C5.grid(column=2,row=8)


Label(root,text='Descripción').grid(row=9,column=0)

Celda01=StringVar()
Celda01=Entry(root,textvariable=Celda01)
Celda01.grid(row=10,column=0)

Label(root,text='Vlr Unidad').grid(row=9,column=2)

Celda02=StringVar()
Celda02.set(0)
Celda02=Entry(root,textvariable=Celda02)
Celda02.grid(row=10,column=2)

Label(root,text='Cantidad').grid(row=9,column=1)

Celda03=StringVar()
Celda03.set(1)
Celda03=Entry(root,textvariable=Celda03)
Celda03.grid(row=10,column=1)

Label(root,text='Descuento: ').grid(row=11,column=0)

Celda04=StringVar()
Celda04.set(0)
Celda04=Entry(root,textvariable=Celda04)
Celda04.grid(row=11,column=1)

Add_celda0_button=Button(root, text='AGREGAR', command=agregarcelda0)
Add_celda0_button.grid(column=1,row=12)

Generate_button=Button(root, text="GENERAR FACTURA", command=factura)
Generate_button.grid(column=1,row=13)
Button(root,text='VER',command=ver).grid(row=14,column=1)
Restar_button=Button(root, text="NUEVA", command=restart_program)
Restar_button.grid(column=1,row=15,pady=30)


root.mainloop()

Exception ignored in: <function ComboboxTip.__del__ at 0x1122f6790>
Traceback (most recent call last):
  File "/var/folders/2j/5plz23md68d4qkdt7ptx58xw0000gn/T/ipykernel_1210/1868948087.py", line 67, in __del__
TypeError: super(type, obj): obj must be an instance or subtype of type


In [6]:
!py -m PyInstaller --windowed MYR_Facturas.py

1046 INFO: PyInstaller: 4.5.1
1046 INFO: Python: 3.9.7
1062 INFO: Platform: Windows-10-10.0.19042-SP0
1078 INFO: wrote C:\Users\simon\Documents\MYR\MYR_Facturas.spec
1093 INFO: UPX is not available.
1109 INFO: Extending PYTHONPATH with paths
['C:\\Users\\simon\\Documents\\MYR', 'C:\\Users\\simon\\Documents\\MYR']
1999 INFO: checking Analysis
1999 INFO: Building Analysis because Analysis-00.toc is non existent
1999 INFO: Initializing module dependency graph...
2015 INFO: Caching module graph hooks...
2218 INFO: Analyzing base_library.zip ...
11312 INFO: Processing pre-find module path hook distutils from 'C:\\Users\\simon\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\PyInstaller\\hooks\\pre_find_module_path\\hook-distutils.py'.
11375 INFO: distutils: retargeting to non-venv dir 'C:\\Users\\simon\\AppData\\Local\\Programs\\Python\\Python39\\lib'
28977 INFO: Caching module dependency graph...
29321 INFO: running Analysis Analysis-00.toc
29447 INFO: Adding Microsoft.Wind